In [65]:
import cv2 as cv
import numpy as np
from tqdm import tqdm

vno = 0

cap = cv.VideoCapture(0)
fps = int(cap.get(cv.CAP_PROP_FPS))
fourcc = cv.VideoWriter_fourcc(*'mp4v')
width = 320
height = 320
vid_cv = []
op_flo_cv = []
flag = False

while(1):
    ret, frame = cap.read()
    frame = cv.flip(frame,1)
    crop = frame[60:-60,320:]
    crop = cv.resize(crop,(320,320))
    #cv.imshow("frame", frame)
    cv.imshow("crop", crop)
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break
    if k == ord('s'):
        if flag == False:
            print("Started recording")
            vid_cv.clear()
            op_flo_cv.clear()
            flag = True
        else:
            flag = False
            if vno%2 == True or vno==0:
                order = "asc"
            else:
                order = "desc"
            #convert to optical flow
            first_frame = vid_cv[0]
            prev_gray = cv.cvtColor(first_frame,cv.COLOR_BGR2GRAY)
            mask = np.zeros_like(first_frame)
            # Sets image saturation to maximum
            mask[..., 1] = 255
            for i in tqdm(range(1, len(vid_cv)-1), desc="Converting to optical flow"):
                gray = cv.cvtColor(vid_cv[i], cv.COLOR_BGR2GRAY)
                flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                # Computes the magnitude and angle of the 2D vectors
                magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
                # Sets image hue according to the optical flow direction
                mask[..., 0] = angle * 180 / np.pi / 2
                # Sets image value according to the optical flow magnitude (normalized)
                mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
                # Converts HSV to RGB (BGR) color representation
                rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
                rgb = np.array(rgb)
                op_flo_cv.append(rgb)
                # Updates previous frame
                prev_gray = gray
            
            #Save video
            video = cv.VideoWriter("data/optical_flow_"+str(order)+"_"+str(int(vno))+".mp4",fourcc, fps, (width, height), 1)
            for i in tqdm(range(0, len(op_flo_cv)), desc="Saving "+str(order)+str(int(vno))):
                video.write(np.array(op_flo_cv[i]))
            video.release()
            
            vno += 0.5
    if flag == True:
        img = np.array(crop)
        vid_cv.append(img)
cv.destroyAllWindows()
cap.release()

Started recording


Saving asc0: 100%|████████████████████████████████████████████████████████████████████| 94/94 [00:00<00:00, 964.65it/s]


Started recording


Saving desc0: 100%|█████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 857.82it/s]
